# Morfología en escala de grises y segmentación mediante el método de línea divisoria de aguas.

En este tutorial estudiaremos la morfología matemática en escala de grises como un caso generalizado de la morfología binaria. Además, emplearemos el método de línea divisoria de aguas para segmentar imágenes, analizaremos cómo se pueden mejorar sus resultados a través de la imposición de mínimos, e identificaremos algunas ventajas y desventajas de este método.

## Morfología en escala de grises

En el tutorial pasado estudiamos las definiciones matemáticas asociadas a las operaciones de morfología binaria. Las 4 operaciones principales pueden resumirse en las siguientes descripciones asociadas a una imagen A y un elemento estructurante B:

1. **Dilatación**: Todas las posibles traslaciones del origen de $\hat{B}$ tales que $A$ y $B$ se sobreponen en al menos un píxel.
1. **Erosión**: Todas las posibles posiciones del origen de $B$ tales que $B$ está completamente contenido en $A$.
1. **Apertura**: La unión de todas las posibles traslaciones de $B$ que caen dentro de $A$.
1. **Clausura**: Todas las posibles traslaciones de $B$ que caen fuera de $A$.

Estas descripciones siguen aplicando para la morfología en escala de grises, es decir en el caso en el que consideramos que las intensidades de la imagen pueden tener más de una magnitud. Sin embargo, en este caso nuestras imágenes y elementos estructurantes deben interpretarse como conjuntos de 3 dimensiones. La intensidad de una imagen con un alto y ancho en el eje $x$ y $y$ puede interpretarse como una profundidad en el eje $z$. Esto es equivalente a la representación de una imagen como un mapa topográfico, donde la intensidad representa la altura del mapa en una coordenada determinada.

La morfología matematica en escala de grises tiene diversos usos. Algunos de ellos pueden ser obtener el gradiente morfológico, suavizado de superficies, obtención de minimos y maximos locales, entre otros. En este tutorial implementaremos algunos de estos algoritmos.

### Erosión en escala de grises
La operación de la erosión en escala de grises no es muy distinta a la erosión en binario. Si recordamos bien la erosión nos decia que el pixel central será 0 si nuestro elemento estructurante B tocaba el fondo, de lo contrario sería 1. Esto se puede interpretar como un filtro minimo, es decír, le asignamos a nuestro pixel central el minimo de la vecindad cubierta por el elemento estructurante. En escala de grises, cuando usamos un elemento estructurante plano, haremos algo muy similar, tomando el minimo de la vecindad donde los pixeles del elemento estructurante están encendidos. 

#### Ejercicio 1:
Modifique la función binary_dilation del tutorial anterior para que acepte imagenes en escala de grises y realice la erosión de la imagen con un elemento estructurante. Considere que el elemento estructurante que entre será binario y que los pixeles encendidos serán 1 y los apagados 0.

In [ ]:
import numpy as np

def binary_dilatation(A, B):
    
    # Obtención de variables auxiliares
    m, n = B.shape
    a, b = (m - 1)//2, (n - 1)//2
    
    # Imagen con borde de ceros que permita central el elemento estructurante en aristas y vertices
    A_pad = np.block([[np.zeros((a, b)),         np.zeros((a, A.shape[1])),    np.zeros((a, b))],
                      [np.zeros((A.shape[0], b)),         A,                   np.zeros((A.shape[0], b))],
                      [np.zeros((a, b)),         np.zeros((a, A.shape[1])),    np.zeros((a, b))]])
    
    # Arreglo en el que se almacenará el resultado de la dilatación sobre la imagen con borde
    result_pad = np.copy(A_pad)
    
    # Para cada pixel del arreglo original
    for x in range(A.shape[0]):
        for y in range(A.shape[1]):
            # Cambio de coordenadas a la imagen con borde
            x_pad = x + a
            y_pad = y + b
            # Si el pixel en el que se centra el elemento estructurante es igual a 1
            if A_pad[x_pad, y_pad] == 1:
                # La imagen resultante tiene intensidad 1 en cada pixel sea 1 en la
                # imagen original o en el elemento estructurante
                result_pad[x_pad - a:x_pad + a + 1, y_pad - b: y_pad + b + 1] = np.logical_or(
                result_pad[x_pad - a:x_pad + a + 1, y_pad - b: y_pad + b + 1], B)
                
    # El resultado corresponde al arreglo sin borde para que se conserven las dimensiones de la imagen original
    result = result_pad[a: -a, b: -b]
    return result

In [ ]:
def gray_erosion(A, B):
    # YOUR CODE HERE
    raise NotImplementedError()
    return result

In [ ]:
import skimage.io as io
from matplotlib import pyplot as plt
from skimage.morphology import disk, erosion, dilation

fig,ax=plt.subplots(1,3,figsize=(10,10))
image=io.imread('hand_x_ray.jpg',as_gray=True)
eroded=gray_erosion(image,disk(5))
sk_eroded=erosion(image,disk(5))

ax[0].imshow(image)
ax[0].axis('off')
ax[0].set_title('Original')
ax[1].imshow(eroded)
ax[1].axis('off')
ax[1].set_title('Erosion propia')
ax[2].imshow(sk_eroded)
ax[2].axis('off')
ax[2].set_title('Erosion skimage')
plt.show()

assert eroded.shape[0]==image.shape[0] and eroded.shape[1]==image.shape[1], 'La erosión debe preservar las dimensiones originales'
assert np.mean(eroded-sk_eroded)<0.01, 'La erosión propia no se parece lo suficiente a la de skimage'

### Erosión geodesica
La erosión geodesica en escala de grises es similar a la realizada en binario, sin embargo, ya no usaremos como mascara una imagen binaria sino otra imagen en escala de grises. En este caso no usaremos la intesección entre la mascara y la erosión sino la unión, es decir, no será un logical or sino un maximo pixel a pixel.

En resumen, el procedimiento a seguir es el siguiente:

>- Se erosiona la semilla
>- Se obtiene el maximo pixel a pixel entre la imagen **original** y la erosión
>- Se repite iterativamente hasta que la iteración anterior sea igual a la actual

**Nota:** En escala de grises no se necesita usar un condicional.

#### Ejercicio 2:
Cree una función que haga la erosión geodesica de una imagen en escala de grises. Puede usar como plantilla la función realizada en el tutorial pasado.

In [ ]:
def Erosion_geodesica_gray(seed,mask,E1,max_iterations):
    'seed (ndarray): Semilla para erosionar'
    'mask (ndarray): Imagen original'
    'E1 (ndarray): Elemento estructurante para erosionar'
    'max_iterations (int): numero maximo de iteraciones'
    # YOUR CODE HERE
    raise NotImplementedError()
    return result,iterations

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(10,10))
image=io.imread('hand_x_ray.jpg',as_gray=True)
clausura,iteraciones=Erosion_geodesica_gray(dilation(image,disk(4)),image,disk(1),1000)
minimos=1-(image-clausura)

ax[0].imshow(image)
ax[0].axis('off')
ax[0].set_title('Original')
ax[1].imshow(clausura)
ax[1].axis('off')
ax[1].set_title('Apertura por reconstrucción')
ax[2].imshow(minimos)
ax[2].axis('off')
ax[2].set_title('Elemento eliminados')
plt.show()

assert np.array_equal(np.max([erosion(clausura),image],axis=0),clausura) or iteraciones==1000, 'Seguramente salió antes de tiempo del while'
assert np.isclose(np.sum(minimos),710006.46,0.01), 'Los minimos obtenidos por la erosión por reconstrucción son erroneos'

### Imposición de minimos
Como vimos con anterioridad, la clausura por reconstrucción elimina elementos de la imagen que son menores a el elemento estructurante y que son oscuros. Estos pueden considerarse como los minimos locales donde el elemento estructurante no alcanza a entrar. Ahora, si deseamos imponer algunos minimos, es decir poner como minimos locales los puntos que determinamos, debemos usar una formula. La formula es la siguiente:

$R_{min(f+a,fm)}^{\varepsilon}(fm)$

Donde fm es la imagen con minimos, min(f+a,fm) es el minimo pixel a pixel entre f+a y fm, $R_{min(f+a,fm)}^{\varepsilon}(fm)$ es la erosión geodesica usando fm como semilla y el minimo entre f+a y fm como mascara. Para implementar esta función siga los siguientes pasos.

>- Agregue a fm el valor de a, el resultado sera f1.
>- Invierta fm, ya que los minimos a imponer son 1 en donde existen y 0 en donde no, debemos sacar el complemento para continuar el procedimiento.
>- Sacamos el minimo pixel a pixel entre f1 y fm. Esta será la mascara.
>- Realizamos la erosión geodesica usando fm como semilla, la mascara y el elemento estructurante E1.

In [ ]:
def Minim_imposicion(f,fm,E1,a,max_iterations):
    'f (ndarray): Imagen original'
    'fm (ndarray): Minimos a imponer'
    'a (float): Valor a agregar a fm'
    'E1 (ndarray): Elemento estructurante para erosion geodesica'
    'max_iterations (int): numero maximo de iteraciones'
    # YOUR CODE HERE
    raise NotImplementedError()
    return R

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(10,10))
image=io.imread('hand_x_ray.jpg',as_gray=True)
minimos_=dilation(minimos>1.2,disk(10))
minim_imposed=Minim_imposicion(image,minimos_,disk(1),0.01,10000)

ax[0].imshow(image)
ax[0].axis('off')
ax[0].set_title('Original')
ax[1].imshow(minim_imposed)
ax[1].axis('off')
ax[1].set_title('Minimos impuestos')
ax[2].imshow(minimos_)
ax[2].axis('off')
ax[2].set_title('Marcadores')
plt.show()

In [ ]:
assert not np.sum(minim_imposed*minimos_), 'Los minimos estan mal puestos'
assert not np.mean(((minimos_*image)+np.min([image,minim_imposed],axis=0))-image), 'La imposición de minimos solo deben alterar las semillas puestas y el resto de minimos, no la imagen original'